In [ ]:
import h5py
import matplotlib.pyplot as plt
import mpl_lego as mplego
import neuropacks
import numpy as np
import os

from pyprojroot import here
from scipy.stats import wilcoxon
from mpl_lego.labels import add_significance_label, bold_text

%matplotlib inline

In [ ]:
mplego.style.use_latex_style()

In [ ]:
data_root = here('data')

In [ ]:
# Import PVC data
pvc_data_path = os.path.join(data_root, 'data_monkey1_gratings.mat')
pvc11 = neuropacks.PVC11(data_path=pvc_data_path)

In [ ]:
# Import PVC fits
pvc_cotu_results = h5py.File(here('fits/pvc11_cotu_fits.h5'), 'r')
pvc_cotula_results = h5py.File(here('fits/pvc11_cotula_fits.h5'), 'r')
b_pvc_cotula = np.array(pvc_cotula_results['b'][:])
modulations_pvc_cotula, _ = pvc11.get_tuning_modulation_and_preference(form='cosine2', tuning_coefs=b_pvc_cotula)

In [ ]:
# Import ECOG data
ecog_data_path = os.path.join(data_root, 'r32_b7.mat')
ecog_grid_path = os.path.join(data_root, 'grdid.mat')
ecog = neuropacks.ECOG(data_path=ecog_data_path, grid_path=ecog_grid_path)

In [ ]:
# Import ECOG fits
ecog_cotu_fits = h5py.File(here('fits/ecog_cotu_fits.h5'), 'r')
ecog_cotula_fits = h5py.File(here('fits/ecog_cotula_fits.h5'), 'r')
b_ecog_cotula = np.array(ecog_cotula_fits['b'][:])
n_electrodes = b_ecog_cotula.shape[0]
modulations_ecog_cotula = np.array(
    [ecog.get_tuning_modulation_and_preference(tuning_coefs=b_ecog_cotula[idx], form='bf')[0]
     for idx in range(n_electrodes)]).ravel()

In [ ]:
"""
Figure 4
"""
width = 5.5
height = 4
fig = plt.figure(figsize=(width, height))

# Preset colors
tu_model_color = 'black'
cotu_model_color = 'gray'
cotula_model_color = 'red'
# Tuning plot settings
tuning_curve_lw = 2
tuning_plot_label_size = 15
# Scatterplot settings
scatter_marker = 'o'
scatter_face_color = 'black'
scatter_edge_color = 'white'
scatter_point_size = 50
scatter_alpha = 0.5
scatter_label_size = 18
scatter_xlabel_pad = 2
scatter_tick_size = 15
# Legend settings
legend_handletextpad = 0.5
legend_handlelength = 1
legend_columnspacing = 0.7
legend_size = 13
# Identity line settings
identity_line_color = 'gray'
identity_line_lw = 3
identity_line_alpha = 0.5
# Inset inset settings
box_width = 0
box_lw = 3
box_marker = 'o'
box_whisker_lw = 0
box_showfliers = False
inset_label_size = 13
inset_tick_size = 12
inset_sig_size = 15
# Subplot label settings
subplot_label_size = 20
subplot_label_bold = True

"""
Figure 4a:
Plot tuning curves, PVC fits
"""
# Figure dimensions
pvc_tcurve_x = 0
pvc_tcurve_y_base = 0
pvc_tcurve_w = 1
pvc_tcurve_h = 1

# Choose example neurons
neuron_ids = np.array([49, 23, 103, 43])

# Iterate over example neurons
for n_idx, neuron in enumerate(neuron_ids):
    # Create axis object
    pvc_tuning_plot = fig.add_axes([pvc_tcurve_x,
                                    pvc_tcurve_y_base + n_idx * (pvc_tcurve_w / height),
                                    pvc_tcurve_w / width,
                                    pvc_tcurve_h / height])

    # Calculate tuning curves
    angles, pvc_tu_tuning = pvc11.get_tuning_curve(
        form='cosine2',
        tuning_coefs=np.mean(pvc_cotu_results['t_cosine2/tuning_coefs'], axis=0)[neuron])
    angles, pvc_cotu_tuning = pvc11.get_tuning_curve(
        form='cosine2',
        tuning_coefs=np.mean(pvc_cotu_results['tc_cosine2_Lasso/tuning_coefs'], axis=0)[neuron])
    angles, pvc_cotula_tuning = pvc11.get_tuning_curve(
        form='cosine2',
        tuning_coefs=b_pvc_cotula[neuron])
    # Plot tuning curves
    pvc_tuning_plot.plot(
        angles,
        pvc_tu_tuning,
        color=tu_model_color,
        linewidth=tuning_curve_lw)
    pvc_tuning_plot.plot(
        angles,
        pvc_cotu_tuning,
        color=cotu_model_color,
        linewidth=tuning_curve_lw)
    pvc_tuning_plot.plot(
        angles,
        pvc_cotula_tuning,
        color=cotula_model_color,
        linewidth=tuning_curve_lw)
    # Set axis bounds
    pvc_tuning_plot.set_xlim([0, 180])
    pvc_tuning_plot.set_xticks([])
    pvc_tuning_plot.set_yticks([])
    # Insert labels
    if n_idx == 0:
        pvc_tuning_plot.set_xlabel(bold_text('Angle'), fontsize=tuning_plot_label_size)
        pvc_tuning_plot.set_ylabel(bold_text('Response'), fontsize=tuning_plot_label_size)

"""
Figure 4b:
Scatter plot of modulations, PVC fits
"""
# Figure dimensions
pvc_scatter_pad = 0.95
pvc_scatter_x = pvc_tcurve_x + pvc_tcurve_w + pvc_scatter_pad
pvc_scatter_y = 0
pvc_scatter_w = 4
pvc_scatter_h = 4
# Create axis object
pvc_scatter = fig.add_axes([pvc_scatter_x / width,
                            pvc_scatter_y,
                            pvc_scatter_w / width,
                            pvc_scatter_h / height])

# Plot modulations
pvc_scatter.scatter(
    pvc_cotu_results['tc_cosine2_Lasso/modulations_mean'][:],
    modulations_pvc_cotula,
    color=scatter_face_color,
    edgecolor=scatter_edge_color,
    marker=scatter_marker,
    alpha=scatter_alpha,
    s=scatter_point_size)

# Plot placeholders for labels
pvc_scatter.plot([], color=tu_model_color, label=bold_text('Tuning Model'))
pvc_scatter.plot([], color=cotu_model_color, label=bold_text('CoTu Model'))
pvc_scatter.plot([], color=cotula_model_color, label=bold_text('CoTuLa Model'))

# Create legend
lgd = pvc_scatter.legend(
    loc=2,
    ncol=1,
    handletextpad=legend_handletextpad,
    handlelength=legend_handlelength,
    columnspacing=legend_columnspacing,
    prop={'size': legend_size})

# Identity line 
pvc_scatter.plot([0.01, 10], [0.01, 10],
                 color=identity_line_color,
                 linewidth=identity_line_lw,
                 alpha=identity_line_alpha)
# Axis bounds, ticks, and scales
pvc_scatter.set_xlim([0.01, 10])
pvc_scatter.set_ylim([0.01, 10])
pvc_scatter.set_xticks([0.01, 0.1, 1, 10])
pvc_scatter.set_yticks([0.01, 0.1, 1, 10])
pvc_scatter.set_xscale('log')
pvc_scatter.set_yscale('log')
# Axis labels
pvc_scatter.set_xlabel(bold_text('CoTu Model'),
                       fontsize=scatter_label_size,
                       labelpad=scatter_xlabel_pad)
pvc_scatter.set_ylabel(bold_text('CoTuLa Model'),
                       fontsize=scatter_label_size)
pvc_scatter.tick_params(labelsize=scatter_tick_size)

"""
Figure 4b, inset:
Boxplot of modulations
"""
# Figure dimensions
pvc_modulations_pad = 2.90
pvc_modulations_x = pvc_scatter_x + pvc_modulations_pad
pvc_modulations_y = 0.1
pvc_modulations_w = 1.
pvc_modulations_h = 1.2
pvc_mod_ax = fig.add_axes([pvc_modulations_x / width,
                           pvc_modulations_y / height,
                           pvc_modulations_w / width,
                           pvc_modulations_h / height])
# Create lists for each modulation set
colors = [tu_model_color, cotu_model_color, cotula_model_color]
mod_markers = ['o', 'D', 'x']
offsets = [0.25, 0, -0.25]
# Iterate over models, plotting the modulations separately
for model_idx, model in enumerate([pvc_cotu_results['t_cosine2/modulations_mean'][:],
                                   pvc_cotu_results['tc_cosine2_Lasso/modulations_mean'][:],
                                   modulations_pvc_cotula]):
    pvc_mod_ax.boxplot(
        x=[model],
        positions=[model_idx],
        widths=box_width,
        boxprops={'linewidth': box_lw, 'color': colors[model_idx]},
        medianprops={'marker': box_marker, 'markersize': 7, 'color': colors[model_idx]},
        whiskerprops={'linewidth': box_whisker_lw, 'color': colors[model_idx]},
        showfliers=box_showfliers)
# Set axis limits, ticks, and scale
pvc_mod_ax.set_xlim([-0.5, 2.5])
pvc_mod_ax.set_ylim([0.1, 10])
pvc_mod_ax.set_xticks([])
pvc_mod_ax.set_yticks([0.1, 1, 10])
pvc_mod_ax.set_yscale('log')
pvc_mod_ax.tick_params(labelsize=inset_tick_size)
# Create label
pvc_mod_ax.set_ylabel(bold_text('Modulations'), fontsize=inset_label_size)
# Add significance labels
add_significance_label(pvc_mod_ax,
                       bounds=[1, 2],
                       label='***',
                       fontsize=inset_sig_size)
add_significance_label(pvc_mod_ax,
                       bounds=[0, 2],
                       label='***',
                       fontsize=inset_sig_size,
                       spacing=0.2)

"""
Figure 4c:
Plot tuning curves, ECOG fits
"""
# Figure dimensions
ecog_tcurve_pad = 1
ecog_tcurve_x = pvc_scatter_x + pvc_scatter_w + ecog_tcurve_pad
ecog_tcurve_y_base = 0
ecog_tcurve_w = 1
ecog_tcurve_h = 1

# Choose example electrodes
electrode_ids = np.array([59, 77, 26, 62])

# Iterate over example electrodes
for e_idx, electrode in enumerate(electrode_ids):
    # Create axis object
    ecog_tuning_plot = fig.add_axes([ecog_tcurve_x / width,
                                     ecog_tcurve_y_base + e_idx * (ecog_tcurve_w / height),
                                     ecog_tcurve_w / width,
                                     ecog_tcurve_h / height])
    # Calculate tuning curves
    frequencies, ecog_tu_tuning = ecog.get_tuning_curve(
        tuning_coefs=np.mean(ecog_cotu_fits['t_bf_lasso/tuning_coefs'], axis=0)[electrode])
    frequencies, ecog_cotu_tuning = ecog.get_tuning_curve(
        tuning_coefs=np.mean(ecog_cotu_fits['tc_bf_lasso/tuning_coefs'], axis=0)[electrode])
    frequencies, ecog_cotula_tuning = ecog.get_tuning_curve(
        tuning_coefs=b_ecog_cotula[electrode])
    # Plot tuning curves
    ecog_tuning_plot.plot(
        frequencies,
        ecog_tu_tuning.ravel() - ecog_tu_tuning.ravel().min(),
        color=tu_model_color,
        linewidth=tuning_curve_lw)
    ecog_tuning_plot.plot(
        frequencies,
        ecog_cotu_tuning.ravel() - ecog_cotu_tuning.ravel().min(),
        color=cotu_model_color,
        linewidth=tuning_curve_lw)
    ecog_tuning_plot.plot(
        frequencies,
        ecog_cotula_tuning.ravel() - ecog_cotula_tuning.ravel().min(),
        color=cotula_model_color,
        linewidth=tuning_curve_lw)
    
    # Set axis bounds
    ecog_tuning_plot.set_xlim([400, 34000])
    ecog_tuning_plot.set_xscale('log')
    ecog_tuning_plot.set_xticks([])
    ecog_tuning_plot.set_yticks([])
    
    # Set axis labels
    if e_idx == 0:
        ecog_tuning_plot.set_xlabel(bold_text('Log Freq.'), fontsize=tuning_plot_label_size)
        ecog_tuning_plot.set_ylabel(bold_text('Response'), fontsize=tuning_plot_label_size)

"""
Figure 4d:
Scatter plot of modulations, ECOG fits
"""
# Figure dimensions
ecog_scatter_pad = pvc_scatter_pad
ecog_scatter_x = ecog_tcurve_x + ecog_tcurve_w + ecog_scatter_pad
ecog_scatter_y = 0
ecog_scatter_w = 4
ecog_scatter_h = 4
# Create axis object
ecog_scatter = fig.add_axes([ecog_scatter_x / width,
                             ecog_scatter_y,
                             ecog_scatter_w / width,
                             ecog_scatter_h / height])

# Plot modulations
ecog_scatter.scatter(
    np.mean(ecog_cotu_fits['tc_bf_lasso/modulations'][:], axis=0),
    modulations_ecog_cotula,
    color=scatter_face_color,
    edgecolor=scatter_edge_color,
    marker=scatter_marker,
    alpha=scatter_alpha,
    s=scatter_point_size)


# Plot placeholders for labels
ecog_scatter.plot([], color=tu_model_color, label=bold_text('Tuning Model'))
ecog_scatter.plot([], color=cotu_model_color, label=bold_text('CoTu Model'))
ecog_scatter.plot([], color=cotula_model_color, label=bold_text('CoTuLa Model'))

# Create legend
lgd = ecog_scatter.legend(
    loc=2,
    ncol=1,
    handletextpad=legend_handletextpad,
    handlelength=legend_handlelength,
    columnspacing=legend_columnspacing,
    prop={'size': legend_size})

# Identity line 
ecog_scatter.plot([0.001, 10], [0.001, 10],
                  color=identity_line_color,
                  linewidth=identity_line_lw,
                  alpha=identity_line_alpha)
# Axis bounds, ticks, and scales
ecog_scatter.set_xlim([0.001, 10])
ecog_scatter.set_ylim([0.001, 10])
ecog_scatter.set_xticks([0.001, 0.01, 0.1, 1, 10])
ecog_scatter.set_yticks([0.001, 0.01, 0.1, 1, 10])
ecog_scatter.set_xscale('log')
ecog_scatter.set_yscale('log')
# Axis labels
ecog_scatter.set_xlabel(bold_text('CoTu Model'),
                        fontsize=scatter_label_size,
                        labelpad=scatter_xlabel_pad)
ecog_scatter.set_ylabel(bold_text('CoTuLa Model'),
                        fontsize=scatter_label_size)
ecog_scatter.tick_params(labelsize=scatter_tick_size)

"""
Figure 4b, inset:
Boxplot of modulations
"""
# Figure dimensions
ecog_modulations_pad = pvc_modulations_pad
ecog_modulations_x = ecog_scatter_x + ecog_modulations_pad
ecog_modulations_y = 0.1
ecog_modulations_w = 1.
ecog_modulations_h = 1.2
ecog_mod_ax = fig.add_axes([ecog_modulations_x / width,
                            ecog_modulations_y / height,
                            ecog_modulations_w / width,
                            ecog_modulations_h / height])
# Create lists for each modulation set
colors = [tu_model_color, cotu_model_color, cotula_model_color]
mod_markers = ['o', 'D', 'x']
offsets = [0.25, 0, -0.25]
# Iterate over models, plotting the modulations separately
for model_idx, model in enumerate([np.mean(ecog_cotu_fits['t_bf_lasso/modulations'][:], axis=0),
                                   np.mean(ecog_cotu_fits['tc_bf_lasso/modulations'][:], axis=0),
                                   modulations_ecog_cotula]):
    ecog_mod_ax.boxplot(
        x=[model],
        positions=[model_idx],
        widths=box_width,
        boxprops={'linewidth': box_lw, 'color': colors[model_idx]},
        medianprops={'marker': box_marker, 'markersize': 7, 'color': colors[model_idx]},
        whiskerprops={'linewidth': box_whisker_lw, 'color': colors[model_idx]},
        showfliers=box_showfliers)
# Set axis limits, ticks, and scale
ecog_mod_ax.set_xlim([-0.5, 2.5])
ecog_mod_ax.set_ylim([0.01, 10])
ecog_mod_ax.set_xticks([])
ecog_mod_ax.set_yticks([0.01, 0.1, 1, 10])
ecog_mod_ax.set_yscale('log')
ecog_mod_ax.tick_params(labelsize=inset_tick_size)
# Create label
ecog_mod_ax.set_ylabel(bold_text('Modulations'), fontsize=inset_label_size)
# Add significance labels
add_significance_label(ecog_mod_ax,
                       bounds=[1, 2],
                       label='***',
                       fontsize=inset_sig_size)
add_significance_label(ecog_mod_ax,
                       bounds=[0, 2],
                       label='***',
                       fontsize=inset_sig_size,
                       spacing=0.2)

# Add subplot labels
mplego.labels.apply_subplot_labels(
    axes=[pvc_tuning_plot, ecog_tuning_plot],
    y=1.2,
    labels=['a', 'c'],
    bold=subplot_label_bold,
    size=subplot_label_size)
mplego.labels.apply_subplot_labels(
    axes=[pvc_scatter, ecog_scatter],
    labels=['b', 'd'],
    bold=subplot_label_bold,
    size=subplot_label_size)

# Save figure
plt.savefig('figure4.pdf', bbox_inches='tight')